# Prepare trait and 16S rRNA gene sequence data

```sh
# Working Directory
cd Bac2fFeature/scripts/01_data_preparation
# Conda environment
conda create --name bac2feature --file ../../environment/bac2feature_packages.yml
conda activate bac2feature

mkdir ../../data/trait ../../silva
```

## Download

### Trait database (Madin et al., 2020)

Downloaded at 2024.06.06
```sh
wget -nc -P ../../data/trait https://raw.githubusercontent.com/bacteria-archaea-traits/bacteria-archaea-traits/master/output/condensed_species_NCBI.csv
```

### SILVA 16S rRNA seqs (version 138.1)

Downloaded at 2024.06.06
```sh
silva_dir="../../data/silva"
# 16S rRNA sequences, multiple sequence alignment, phylogenetic tree, and metadata from SILVA release. 138
## Full 16S rRNA sequences
wget -nc -P ${silva_dir} https://www.arb-silva.de/fileadmin/silva_databases/release_138.1/Exports/SILVA_138.1_SSURef_NR99_tax_silva.fasta.gz
## Multiple sequence alignment
wget -nc -P ${silva_dir} https://www.arb-silva.de/fileadmin/silva_databases/release_138.1/Exports/SILVA_138.1_SSURef_NR99_tax_silva_full_align_trunc.fasta.gz
## Phylogenetic tree derived from 16S rRNA seqs
wget -nc -P ${silva_dir} https://www.arb-silva.de/fileadmin/silva_databases/release_138.1/user/guide-trees/SILVA_138.1_SSURef_NR99.accessions.ntree.gz
## Metadata
wget -nc -P ${silva_dir} https://www.arb-silva.de/fileadmin/silva_databases/release_138.1/Exports/full_metadata/SILVA_138.1_SSURef_Nr99.full_metadata.gz
# Unpack
find ${silva_dir} -type f -name "*.gz" -exec unpigz -k {} \;
```